## [목차]
### 데이터 분석 프로세스
- 데이터 수집, 저장소, 정리
- (선택) 분석용 데이터 분석, 정제(결측치), 라벨링, 증강
- (전처리) 전처리 데이터, 마이닝, 시각화, 특징 추출
- (변환) 데이터 변환, Train/Test set
- (AI모델) 모델
- 활용
### 1. 데이터 정제
  - Data Cleansing - 정제1 : 결측치 처리 (결측치 포함여부, 발생원인 파악, 비율 확인, 처리하기)
  - Data Cleansing - 정제2 : 이상치 처리 (이상치 포함여부(백분위수), 발생원인 파악, 처리방법 결정(삭제/대체), 처리하기)
### 2. 데이터 변환
  - 통합 : 조인(기준_key을 두고 합치기) / 병합(이어 붙이기)
  - 변환 : 파생변수/원핫인코딩, 정렬(소팅), 피봇테이블, 행렬전환, 행 변경 등
    - groupby : 열을 기준으로 묶어서 agg 연산
    - pivot table : 행과 열 기준으로 그룹화하여 연산, 통계화
  - 스케일러(Scaler)
    - Standard Scaler
    - Min-Max Scaler
    - Robust Scaler
  - 축소 : 제거, 대체, 중복값, 상관관계 기반 중복속성 판단
  - df.duplicated() → df.drop_duplicates(inplace=True)
  - Redundant attributes
    - 상관도 : 상관계수(Correlation coefficient), 공분산, 피어슨 상관계수
### 3. 시계열 데이터 전처리
- 시계열 데이터 : 시간 순서로 배열된 데이터
- N * M * Z (배치 * 행 * 열)
- 슬라이딩 윈도우 : 예측타겟에 사용할 데이터의 묶음 생성
- 결측치를 버리거나 선형 보간법 처리

## 데이터 전처리 연습

**`pd.read_csv`를 이용하여 titanic 데이터를 로드합니다.**

In [25]:
data=pd.read_csv('./titanic_train.csv')

**data의 상위 5개 row를 확인합니다.**

In [26]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**각 컬럼의 Null 갯수를 확인합니다.**

In [27]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [28]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [31]:
# 빈 나이를 채우고 싶은데 어떻게 채울것이냐?

**`Age`의 max, min, mean 값을 확인합니다.**


In [32]:
print(f"{data['Age'].max()}")
print(f"{data['Age'].min()}")
print(f"{data['Age'].mean()}")

80.0
0.42
29.69911764705882


**`Name` 컬럼에서 initial 부분만 가져와 `data`의 `Initial`컬럼에 저장합니다.**

In [36]:
data.Name.str.extract('Harris')  # capture groups → 정규표현식 : 괄호 형태로 표현

ValueError: pattern contains no capture groups

In [37]:
data.Name.str.extract('(Harris)')

,0
0,Harris
1,NaN
2,NaN
3,NaN
4,NaN
...,...
886,NaN
887,NaN
888,NaN
889,NaN


In [39]:
data.Name.str.extract('([A-Za-z]+)\.')  # 이니셜을 모으자 . 앞에 단어들로

,0
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr
...,...
886,Rev
887,Miss
888,Miss
889,Mr


In [33]:
data['Initial']=0

for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.')  # 정규표현식 이해에 대한 공부가 좀 필요함
    
data['Initial']

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886     Rev
887    Miss
888    Miss
889      Mr
890      Mr
Name: Initial, Length: 891, dtype: object

#### 정규표현식
- 괄호 안은 group 이다
- 대괄호 안에 있는 것들 : A-Za-z 영어문자 아무거나
- '+' : 한 개 이상 붙어있는 영어문자 골라
- '\.' : 마침표 → 그냥 . 이면 문자 한 개를 뜻하기 때문에 마침표라는 의미를 가지려면 '\.'

In [41]:
# 정규표현식
# 괄호 안은 group 이다
# 대괄호 안에 있는 것들 : A-Za-z 영어문자 아무거나
# + : 한 개 이상 붙어있는 영어문자 골라
# \. : 마침표 → 그냥 . 이면 문자 한 개를 뜻하기 때문에 마침표라는 의미를 가지려면 '\.'

**Sex 별 Initial 수를 확인합니다.**

`pd.crosstab`을 이용하여 `data.Initial`, `data.Sex` 의 교차표를 구합니다.

In [42]:
pd.crosstab(data.Initial, data.Sex).T

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


In [43]:
pd.crosstab(data.Sex, data.Initial)

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


**`Initial`의 철자가 틀린 것들을 올바르게 변환합니다.**

In [44]:
data['Initial'].replace(
    ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],  # 바꾸고자 하는 것들
    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],  # 각각 대응되어서 바꿀 것들
    inplace=True
    )

In [49]:
pd.crosstab(data.Sex, data.Initial)

Initial,Master,Miss,Mr,Mrs,Other
Sex,,,,,
female,0,186,1,127,0
male,40,0,528,0,9


In [50]:
# 주어진 이름으로부터 이니셜 뽑아내고 사용될 형태로 재가공 수행

In [51]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Initial
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Other
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr


**[TODO] `Initial`별로 `Age`의 평균을 출력합니다.**

In [56]:
data.groupby('Initial').apply(lambda x: print(x))

     PassengerId  Survived  Pclass  \
7              8         0       3   
16            17         0       3   
50            51         0       3   
59            60         0       3   
63            64         0       3   
65            66         1       3   
78            79         1       2   
125          126         1       3   
159          160         0       3   
164          165         0       3   
165          166         1       3   
171          172         0       3   
176          177         0       3   
182          183         0       3   
183          184         1       2   
193          194         1       2   
261          262         1       3   
278          279         0       3   
305          306         1       1   
340          341         1       2   
348          349         1       3   
386          387         0       3   
407          408         1       2   
445          446         1       1   
480          481         0       3   
489         

""


In [58]:
data.groupby('Initial')['Age'].agg('mean')

Initial
Master     4.574167
Miss      21.860000
Mr        32.739609
Mrs       35.981818
Other     45.888889
Name: Age, dtype: float64

**[TODO] `Age`가 null인 경우 `Initials`별로 `Age`의 평균을 참고하여 값을 채웁니다.**

In [59]:
data.loc[(data.Age.isnull()) & (data.Initial == 'Master'), 'Age'] = 5  # & 교집합, | 합집합
data.loc[(data.Age.isnull()) & (data.Initial == 'Miss'), 'Age'] = 22  # 나이니까 반올림
data.loc[(data.Age.isnull()) & (data.Initial == 'Mr'), 'Age'] = 33
data.loc[(data.Age.isnull()) & (data.Initial == 'Mrs'), 'Age'] = 36
data.loc[(data.Age.isnull()) & (data.Initial == 'Other'), 'Age'] = 46

**[TODO] `Age`에 null 값이 있는지 확인합니다.**

In [60]:
data.Age.isnull().any() 

False

In [61]:
data.isnull().sum() 

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Initial          0
dtype: int64

가족 구성원의 수를 담은 컬럼을 추가합니다.

In [62]:
data['FamilySize'] = data.SibSp + data.Parch + 1

혼자 탑승했는지에 대한 유무를 담은 컬럼을 추가합니다.

In [63]:
data['IsAlone'] = 1 
data['IsAlone'].loc[data['FamilySize'] > 1] = 0 

C:\Users\king5\AppData\Local\Temp\ipykernel_4904\1316621672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['IsAlone'].loc[data['FamilySize'] > 1] = 0


In [64]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Initial,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,2,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,1,1


텍스트 데이터이지만 카테고리컬 한 의미를 가진 컬럼들을 정수형으로 변환한다.

In [65]:
label = LabelEncoder()  # 수치형으로 일단 바꿔놓고, 뒤에서 원 핫 인코딩 할 때 쓰기 쉽도록 변환해준다

data['Sex_Code'] = label.fit_transform(data['Sex'])
data['Embarked_Code'] = label.fit_transform(data['Embarked'])
data['Initial_Code'] = label.fit_transform(data['Initial'])

In [66]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Initial,FamilySize,IsAlone,Sex_Code,Embarked_Code,Initial_Code
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.2500,NaN,S,Mr,2,0,1,2,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.2833,C85,C,Mrs,2,0,0,0,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1,1,0,2,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.1000,C123,S,Mrs,2,0,0,2,3
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.0500,NaN,S,Mr,1,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.00,0,0,211536,13.0000,NaN,S,Other,1,1,1,2,4
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.00,0,0,112053,30.0000,B42,S,Miss,1,1,0,2,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.86,1,2,W./C. 6607,23.4500,NaN,S,Miss,4,0,0,2,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.00,0,0,111369,30.0000,C148,C,Mr,1,1,1,0,2
